In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv(r"C:\Users\rosem\Downloads\FINAL_PROJECT_WORKS\CSV_files\cleaned_data.csv")
data

,UTC_Time,Date,Region,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,Solar_Gen,CO2_Factor_Coal,CO2_Factor_Gas,CO2_Emissions_Coal,CO2_Emissions_Gas,CO2_Total_Emissions,CO2_Intensity_Generated,CO2_Intensity_Consumed
0,2018-07-01 08:00:00,2018-07-01,CAL,1,26061.0,17023.0,-6508.0,25443.0,7.0,7675.0,2267.0,2560.0,0.0,2.23,0.93,7.07,3221.51,3373.68,0.44,0.38
1,2018-07-01 09:00:00,2018-07-01,CAL,2,24604.0,15411.0,-6710.0,24029.0,9.0,5879.0,2261.0,2584.0,0.0,2.23,0.93,9.09,2467.84,2623.17,0.37,0.33
2,2018-07-01 10:00:00,2018-07-01,CAL,3,23554.0,14680.0,-6840.0,22984.0,6.0,5635.0,2258.0,2506.0,0.0,2.23,0.92,6.06,2362.67,2511.83,0.38,0.33
3,2018-07-01 11:00:00,2018-07-01,CAL,4,22843.0,14362.0,-6843.0,22343.0,8.0,5405.0,2257.0,2446.0,0.0,2.23,0.93,8.08,2268.50,2419.01,0.37,0.32
4,2018-07-01 12:00:00,2018-07-01,CAL,5,22538.0,14155.0,-6928.0,22108.0,8.0,5592.0,2260.0,2609.0,0.0,2.23,0.93,8.08,2350.03,2498.24,0.39,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525469,2023-07-01 01:00:00,2023-06-30,TEX,20,73862.0,73920.0,58.0,75075.0,11013.0,36589.0,4642.0,115.0,2237.0,2.33,0.91,11662.81,15069.97,26833.94,0.80,0.80
525470,2023-07-01 02:00:00,2023-06-30,TEX,21,70570.0,70337.0,-232.0,71706.0,10632.0,36722.0,4643.0,133.0,264.0,2.34,0.91,11264.12,15132.49,26484.94,0.83,0.83
525471,2023-07-01 03:00:00,2023-06-30,TEX,22,67978.0,67953.0,-25.0,69197.0,10592.0,35912.0,4643.0,76.0,0.0,2.33,0.91,11217.49,14802.15,26102.08,0.85,0.85
525472,2023-07-01 04:00:00,2023-06-30,TEX,23,64727.0,64768.0,42.0,65551.0,10055.0,32700.0,4645.0,10.0,0.0,2.33,0.91,10637.39,13466.76,24188.75,0.82,0.82


In [4]:
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Create seasonal and time-based features
data['Month'] = data['Date'].dt.month                           # 1 to 12
data['DayOfWeek'] = data['Date'].dt.dayofweek                   # 0 = Monday, ..., 6 = Sunday
data['Is_Weekend'] = data['DayOfWeek'].isin([5, 6]).astype(int) # 1 if Saturday or Sunday
data['DayOfYear'] = data['Date'].dt.dayofyear                   # 1 to 365 (or 366)
data['WeekOfYear'] = data['Date'].dt.isocalendar().week         # 1 to 52 (or 53)

# (Optional) Check your new columns
print(data[['Date', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear']].head())

        Date  Month  DayOfWeek  Is_Weekend  DayOfYear  WeekOfYear
0 2018-07-01      7          6           1        182          26
1 2018-07-01      7          6           1        182          26
2 2018-07-01      7          6           1        182          26
3 2018-07-01      7          6           1        182          26
4 2018-07-01      7          6           1        182          26


In [5]:
# Ensure the 'Date' column is in datetime format
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Sort the data by date just in case
data = data.sort_values(by='Date').reset_index(drop=True)

# Create a complete date range from start to end
full_range = pd.date_range(start=data['Date'].min(), end=data['Date'].max(), freq='H')  # Change freq to 'D' for daily data

# Find missing dates
missing_dates = full_range.difference(data['Date'])

# Show result
if missing_dates.empty:
    print("✅ No missing timestamps — your data is continuous.")
else:
    print(f"❌ {len(missing_dates)} missing timestamps found.")
    print("Sample missing dates:")
    print(missing_dates[:10])  # show first 10

❌ 41975 missing timestamps found.
Sample missing dates:
DatetimeIndex(['2018-07-01 01:00:00', '2018-07-01 02:00:00',
               '2018-07-01 03:00:00', '2018-07-01 04:00:00',
               '2018-07-01 05:00:00', '2018-07-01 06:00:00',
               '2018-07-01 07:00:00', '2018-07-01 08:00:00',
               '2018-07-01 09:00:00', '2018-07-01 10:00:00'],
              dtype='datetime64[ns]', freq=None)


In [6]:
import pandas as pd

# Load your dataset
data['Date'] = pd.to_datetime(data['Date'])

# Create an empty list to collect per-region reindexed data
region_dfs = []

# Get the list of unique regions
regions = data['Region'].unique()

# Loop through each region
for region in regions:
    # Filter data for that region
    region_data = data[data['Region'] == region].copy()

    # Generate complete hourly range for that region
    full_range = pd.date_range(start=region_data['Date'].min(), end=region_data['Date'].max(), freq='H')

    # Group and average numeric values for duplicate timestamps
    numeric_cols = region_data.select_dtypes(include='number').columns
    grouped = region_data.groupby('Date')[numeric_cols].mean().reset_index()

    # Reindex to full hourly timestamps
    reindexed = grouped.set_index('Date').reindex(full_range)
    reindexed.index.name = 'Date'

    # Fill missing region label
    reindexed['Region'] = region

    # Append to the list
    region_dfs.append(reindexed.reset_index())

# Combine all region DataFrames
data_full = pd.concat(region_dfs, ignore_index=True)

# Optional: Sort by Region and Date
data_full = data_full.sort_values(by=['Region', 'Date'])

# Show result
print("✅ Data successfully reindexed per region with missing hours filled as NaN.")
print(data_full.head())

✅ Data successfully reindexed per region with missing hours filled as NaN.
                 Date  Hour    Demand  Net_Generation  Total_Interchange  \
0 2018-07-01 00:00:00  12.5  27455.25    19785.416667       -6089.333333   
1 2018-07-01 01:00:00   NaN       NaN             NaN                NaN   
2 2018-07-01 02:00:00   NaN       NaN             NaN                NaN   
3 2018-07-01 03:00:00   NaN       NaN             NaN                NaN   
4 2018-07-01 04:00:00   NaN       NaN             NaN                NaN   

   Forecasted_Demand  Coal_Gen      Gas_Gen  Nuclear_Gen  Hydro_Gen  ...  \
0       26689.791667  8.291667  7114.541667      2261.25    2735.25  ...   
1                NaN       NaN          NaN          NaN        NaN  ...   
2                NaN       NaN          NaN          NaN        NaN  ...   
3                NaN       NaN          NaN          NaN        NaN  ...   
4                NaN       NaN          NaN          NaN        NaN  ...   

   CO2_Emis

In [7]:
data_full.interpolate(method='linear', inplace=True)

In [8]:
data = data_full.copy()

In [9]:
# Ensure the 'Date' column is in datetime format
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Sort the data by date just in case
data = data.sort_values(by='Date').reset_index(drop=True)

# Create a complete date range from start to end
full_range = pd.date_range(start=data['Date'].min(), end=data['Date'].max(), freq='H')  # Change freq to 'D' for daily data

# Find missing dates
missing_dates = full_range.difference(data['Date'])

# Show result
if missing_dates.empty:
    print("✅ No missing timestamps — your data is continuous.")
else:
    print(f"❌ {len(missing_dates)} missing timestamps found.")
    print("Sample missing dates:")
    print(missing_dates[:10])  # show first 10

✅ No missing timestamps — your data is continuous.


In [10]:
def assign_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

data['Season'] = data['Month'].apply(assign_season)
data = pd.get_dummies(data, columns=['Season'])  # One-hot encode Season
data

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Month,DayOfWeek,Is_Weekend,DayOfYear,WeekOfYear,Region,Season_Autumn,Season_Spring,Season_Summer,Season_Winter
0,2018-07-01,12.5,27455.250000,19785.416667,-6089.333333,26689.791667,8.291667,7114.541667,2261.250000,2735.250000,...,7.0,6.0,1.0,182.0,26.0,CAL,False,False,True,False
1,2018-07-01,12.5,114937.291667,118965.958333,3982.875000,114216.583333,38975.375000,38177.250000,33496.083333,2257.041667,...,7.0,6.0,1.0,182.0,26.0,MIDA,False,False,True,False
2,2018-07-01,12.5,17965.500000,15979.708333,-1985.791667,17863.750000,342.833333,9701.791667,3946.250000,862.166667,...,7.0,6.0,1.0,182.0,26.0,NE,False,False,True,False
3,2018-07-01,12.5,30623.583333,29623.416667,-1018.833333,28525.750000,3199.208333,21023.250000,3779.500000,31.291667,...,7.0,6.0,1.0,182.0,26.0,FLA,False,False,True,False
4,2018-07-01,12.5,39135.541667,43793.625000,9090.333333,37734.000000,9165.000000,7112.875000,986.375000,16349.708333,...,7.0,6.0,1.0,182.0,26.0,NW,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525223,2023-06-30,12.5,36658.541667,36016.250000,-641.125000,35350.541667,2432.458333,23954.958333,3683.875000,34.833333,...,6.0,4.0,0.0,181.0,26.0,FLA,False,False,True,False
525224,2023-06-30,12.5,40239.916667,40975.166667,735.333333,42000.375000,13035.541667,13727.625000,1968.833333,1565.833333,...,6.0,4.0,0.0,181.0,26.0,CENT,False,False,True,False
525225,2023-06-30,12.5,29410.166667,29558.875000,144.708333,30887.791667,7983.583333,6982.208333,11876.166667,368.083333,...,6.0,4.0,0.0,181.0,26.0,CAR,False,False,True,False
525226,2023-06-30,12.5,14294.583333,11865.291667,-2429.291667,14351.250000,0.166667,7797.666667,2040.500000,1186.166667,...,6.0,4.0,0.0,181.0,26.0,NE,False,False,True,False


In [11]:
data['Demand_Prev_Hour'] = data['Demand'].shift(1)
data['Demand_Yesterday_Same_Hour'] = data['Demand'].shift(24)
data['Demand_Last_Week_Same_Hour'] = data['Demand'].shift(168)

In [12]:
data['Rolling_Mean_3H'] = data['Demand'].rolling(window=3).mean()
data['Rolling_Mean_24H'] = data['Demand'].rolling(window=24).mean()

# ========= 4. Energy Mix Ratios =========
data['Total_Gen'] = data[['Coal_Gen', 'Gas_Gen', 'Nuclear_Gen', 'Hydro_Gen', 'Solar_Gen']].sum(axis=1)

data['Renewable_Pct'] = ((data['Hydro_Gen'] + data['Solar_Gen']) / data['Total_Gen']) * 100
data['Fossil_Pct'] = ((data['Coal_Gen'] + data['Gas_Gen']) / data['Total_Gen']) * 100

# ========= 5. Final Cleanup =========
# Drop rows with NaNs created from rolling or lag
data = data.dropna().reset_index(drop=True)
data

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Season_Summer,Season_Winter,Demand_Prev_Hour,Demand_Yesterday_Same_Hour,Demand_Last_Week_Same_Hour,Rolling_Mean_3H,Rolling_Mean_24H,Total_Gen,Renewable_Pct,Fossil_Pct
0,2018-07-01 16:00:00,12.500000,32668.750000,31610.944444,-1129.861111,30188.944444,3395.680556,22638.305556,3774.250000,31.569444,...,True,False,39515.708333,36751.798611,27455.250000,33690.347222,42616.980698,30126.555556,1.056607,86.415409
1,2018-07-01 16:00:00,12.666667,98191.374396,95908.335749,-3872.722222,101822.126812,47810.253623,29087.126812,11639.043478,1224.783816,...,True,False,32668.750000,32413.104167,114937.291667,56791.944243,45357.741958,89860.721014,1.473722,85.573963
2,2018-07-01 17:00:00,12.500000,31560.368056,33623.656250,2208.244792,17062.499167,9879.272569,16289.060764,5835.713542,1332.260417,...,True,False,98191.374396,28707.666667,17965.500000,54140.164151,45476.604516,33652.527778,4.898535,77.760380
3,2018-07-01 17:00:00,12.500000,28976.041667,21138.303819,-6528.677083,28386.220486,8.763889,7725.951389,2261.072917,3089.741319,...,True,False,31560.368056,97906.322162,30623.583333,52909.261373,42604.509495,17521.203125,42.950332,44.144887
4,2018-07-01 17:00:00,12.500000,8551.005208,9138.222222,586.211806,8408.137153,1511.822917,3661.506944,2692.883681,658.407986,...,True,False,28976.041667,31128.128472,39135.541667,23029.138310,41663.796026,8965.375000,12.260072,57.703441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-30 00:00:00,12.500000,36658.541667,36016.250000,-641.125000,35350.541667,2432.458333,23954.958333,3683.875000,34.833333,...,True,False,102607.250000,36775.541667,37477.541667,53861.175000,43957.871934,32352.291667,7.050505,81.562743
525056,2023-06-30 00:00:00,12.500000,40239.916667,40975.166667,735.333333,42000.375000,13035.541667,13727.625000,1968.833333,1565.833333,...,True,False,36658.541667,15892.381354,14364.875000,59835.236111,44972.352572,30351.791667,5.336725,88.176563
525057,2023-06-30 00:00:00,12.500000,29410.166667,29558.875000,144.708333,30887.791667,7983.583333,6982.208333,11876.166667,368.083333,...,True,False,40239.916667,64836.493056,100408.375000,35436.208333,43496.255639,28434.000000,5.599077,52.633438
525058,2023-06-30 00:00:00,12.500000,14294.583333,11865.291667,-2429.291667,14351.250000,0.166667,7797.666667,2040.500000,1186.166667,...,True,False,29410.166667,14304.625000,21558.561944,27981.555556,43495.837236,11185.333333,12.042556,69.714805


In [13]:
data.dtypes

Date                          datetime64[ns]
Hour                                 float64
Demand                               float64
Net_Generation                       float64
Total_Interchange                    float64
Forecasted_Demand                    float64
Coal_Gen                             float64
Gas_Gen                              float64
Nuclear_Gen                          float64
Hydro_Gen                            float64
Solar_Gen                            float64
CO2_Factor_Coal                      float64
CO2_Factor_Gas                       float64
CO2_Emissions_Coal                   float64
CO2_Emissions_Gas                    float64
CO2_Total_Emissions                  float64
CO2_Intensity_Generated              float64
CO2_Intensity_Consumed               float64
Month                                float64
DayOfWeek                            float64
Is_Weekend                           float64
DayOfYear                            float64
WeekOfYear

In [14]:
data = data.round(3)
data

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Season_Summer,Season_Winter,Demand_Prev_Hour,Demand_Yesterday_Same_Hour,Demand_Last_Week_Same_Hour,Rolling_Mean_3H,Rolling_Mean_24H,Total_Gen,Renewable_Pct,Fossil_Pct
0,2018-07-01 16:00:00,12.500,32668.750,31610.944,-1129.861,30188.944,3395.681,22638.306,3774.250,31.569,...,True,False,39515.708,36751.799,27455.250,33690.347,42616.981,30126.556,1.057,86.415
1,2018-07-01 16:00:00,12.667,98191.374,95908.336,-3872.722,101822.127,47810.254,29087.127,11639.043,1224.784,...,True,False,32668.750,32413.104,114937.292,56791.944,45357.742,89860.721,1.474,85.574
2,2018-07-01 17:00:00,12.500,31560.368,33623.656,2208.245,17062.499,9879.273,16289.061,5835.714,1332.260,...,True,False,98191.374,28707.667,17965.500,54140.164,45476.605,33652.528,4.899,77.760
3,2018-07-01 17:00:00,12.500,28976.042,21138.304,-6528.677,28386.220,8.764,7725.951,2261.073,3089.741,...,True,False,31560.368,97906.322,30623.583,52909.261,42604.509,17521.203,42.950,44.145
4,2018-07-01 17:00:00,12.500,8551.005,9138.222,586.212,8408.137,1511.823,3661.507,2692.884,658.408,...,True,False,28976.042,31128.128,39135.542,23029.138,41663.796,8965.375,12.260,57.703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-30 00:00:00,12.500,36658.542,36016.250,-641.125,35350.542,2432.458,23954.958,3683.875,34.833,...,True,False,102607.250,36775.542,37477.542,53861.175,43957.872,32352.292,7.051,81.563
525056,2023-06-30 00:00:00,12.500,40239.917,40975.167,735.333,42000.375,13035.542,13727.625,1968.833,1565.833,...,True,False,36658.542,15892.381,14364.875,59835.236,44972.353,30351.792,5.337,88.177
525057,2023-06-30 00:00:00,12.500,29410.167,29558.875,144.708,30887.792,7983.583,6982.208,11876.167,368.083,...,True,False,40239.917,64836.493,100408.375,35436.208,43496.256,28434.000,5.599,52.633
525058,2023-06-30 00:00:00,12.500,14294.583,11865.292,-2429.292,14351.250,0.167,7797.667,2040.500,1186.167,...,True,False,29410.167,14304.625,21558.562,27981.556,43495.837,11185.333,12.043,69.715


In [15]:
# Convert all boolean columns to integers (True → 1, False → 0)
bool_cols = data.select_dtypes(include='bool').columns
data[bool_cols] = data[bool_cols].astype(int)
data

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Season_Summer,Season_Winter,Demand_Prev_Hour,Demand_Yesterday_Same_Hour,Demand_Last_Week_Same_Hour,Rolling_Mean_3H,Rolling_Mean_24H,Total_Gen,Renewable_Pct,Fossil_Pct
0,2018-07-01 16:00:00,12.500,32668.750,31610.944,-1129.861,30188.944,3395.681,22638.306,3774.250,31.569,...,1,0,39515.708,36751.799,27455.250,33690.347,42616.981,30126.556,1.057,86.415
1,2018-07-01 16:00:00,12.667,98191.374,95908.336,-3872.722,101822.127,47810.254,29087.127,11639.043,1224.784,...,1,0,32668.750,32413.104,114937.292,56791.944,45357.742,89860.721,1.474,85.574
2,2018-07-01 17:00:00,12.500,31560.368,33623.656,2208.245,17062.499,9879.273,16289.061,5835.714,1332.260,...,1,0,98191.374,28707.667,17965.500,54140.164,45476.605,33652.528,4.899,77.760
3,2018-07-01 17:00:00,12.500,28976.042,21138.304,-6528.677,28386.220,8.764,7725.951,2261.073,3089.741,...,1,0,31560.368,97906.322,30623.583,52909.261,42604.509,17521.203,42.950,44.145
4,2018-07-01 17:00:00,12.500,8551.005,9138.222,586.212,8408.137,1511.823,3661.507,2692.884,658.408,...,1,0,28976.042,31128.128,39135.542,23029.138,41663.796,8965.375,12.260,57.703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-30 00:00:00,12.500,36658.542,36016.250,-641.125,35350.542,2432.458,23954.958,3683.875,34.833,...,1,0,102607.250,36775.542,37477.542,53861.175,43957.872,32352.292,7.051,81.563
525056,2023-06-30 00:00:00,12.500,40239.917,40975.167,735.333,42000.375,13035.542,13727.625,1968.833,1565.833,...,1,0,36658.542,15892.381,14364.875,59835.236,44972.353,30351.792,5.337,88.177
525057,2023-06-30 00:00:00,12.500,29410.167,29558.875,144.708,30887.792,7983.583,6982.208,11876.167,368.083,...,1,0,40239.917,64836.493,100408.375,35436.208,43496.256,28434.000,5.599,52.633
525058,2023-06-30 00:00:00,12.500,14294.583,11865.292,-2429.292,14351.250,0.167,7797.667,2040.500,1186.167,...,1,0,29410.167,14304.625,21558.562,27981.556,43495.837,11185.333,12.043,69.715


In [16]:
data = data.sort_values(['Region', 'Date'])
data = data.groupby('Region').apply(lambda group: group.ffill().bfill()).reset_index(drop=True)
data

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Season_Summer,Season_Winter,Demand_Prev_Hour,Demand_Yesterday_Same_Hour,Demand_Last_Week_Same_Hour,Rolling_Mean_3H,Rolling_Mean_24H,Total_Gen,Renewable_Pct,Fossil_Pct
0,2018-07-01 17:00:00,12.5,28976.042,21138.304,-6528.677,28386.220,8.764,7725.951,2261.073,3089.741,...,1,0,31560.368,97906.322,30623.583,52909.261,42604.509,17521.203,42.950,44.145
1,2018-07-01 18:00:00,12.5,29065.500,21217.885,-6554.521,28486.010,8.792,7761.917,2261.062,3110.594,...,1,0,19035.781,36973.722,30187.366,26585.677,41847.195,17577.073,42.927,44.209
2,2018-07-01 19:00:00,12.5,29154.958,21297.467,-6580.365,28585.800,8.819,7797.882,2261.052,3131.446,...,1,0,120405.354,28886.583,96196.009,60325.969,45629.697,17632.943,42.904,44.273
3,2018-07-01 20:00:00,12.5,29244.417,21377.049,-6606.208,28685.590,8.847,7833.847,2261.042,3152.299,...,1,0,8629.271,31655.750,35642.181,23684.576,48609.446,17688.812,42.881,44.337
4,2018-07-01 21:00:00,12.5,29333.875,21456.630,-6632.052,28785.380,8.875,7869.812,2261.031,3173.151,...,1,0,31562.326,29065.500,8211.854,30046.873,49142.567,17744.682,42.858,44.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-29 20:00:00,12.5,64967.778,65025.979,58.042,65732.083,8989.486,27552.910,4647.181,32.896,...,1,0,40554.444,14324.708,37711.542,67747.671,49409.849,46077.069,10.607,79.307
525056,2023-06-29 21:00:00,12.5,64902.135,64963.474,61.177,65699.302,8993.729,27593.401,4647.292,33.224,...,1,0,31739.526,22046.601,65821.128,37511.137,46137.266,46106.208,10.566,79.354
525057,2023-06-29 22:00:00,12.5,64836.493,64900.969,64.312,65666.521,8997.972,27633.892,4647.403,33.552,...,1,0,15892.381,31801.521,21450.109,39168.139,45454.091,46135.347,10.526,79.401
525058,2023-06-29 23:00:00,12.5,64770.851,64838.464,67.448,65633.740,9002.215,27674.384,4647.514,33.880,...,1,0,22263.507,97828.229,41419.396,42477.179,39779.804,46164.486,10.485,79.448


In [17]:
# data.to_csv(r"C:\Users\rosem\Downloads\FINAL_PROJECT_WORKS\CSV_files\featue_engg_data1.csv", index = False)

In [18]:
# data.to_excel(r"C:\Users\rosem\Downloads\FINAL_PROJECT_WORKS\CSV_files\featue_engg_data_xlsx1.xlsx", index = False)